In [75]:
import numpy as np
import pandas as pd

class LangId:
    def __init__(self,smooth = 0.5):
        self.smooth = smooth
        alpha_dict = {"a":0, "b":0, "c":0, "d":0, "e":0, "f":0, "g":0, "h":0, "i":0, "j":0, "k":0, "l":0, "m":0, 
        "n":0, "o":0, "p":0, "q":0, "r":0, "s":0, "t":0, "u":0, "v":0, "w":0, "x":0, "y":0, "z":0, " ":0}
        self.alpha = pd.DataFrame(alpha_dict,index=['e', 'j', 's']) 
        self.path = "../languageID/"
        self.train = []
        self.test = []
        self.lang = np.array(['e','j','s'])
        self.prior = np.array([0,0,0]).astype('float')      #before smooth
        self.theta = [np.array(27), np.array(27), np.array(27)]   # e j s theta
    
    def init_sets(self, train_size = [10,10,10]):
        for i, lang in enumerate(self.lang):   #e j s
            for j in range(0,20):
                if(j < train_size[i]):
                    self.train.append(lang+str(j)+".txt")
                else:
                    self.test.append(lang+str(j)+".txt")
    
    # ---------- Q1: Estimate the prior probabilities ---------- #            
    def get_prior(self):
        e = j = s = 0
        for set in self.train:
            if set.find('e') != -1:
                e += 1
            elif set.find('j') != -1:
                j += 1
            elif set.find('s') != -1:
                s += 1
        denominator = e + j + s + (3*self.smooth)
        self.prior[0] = (e+self.smooth)/denominator
        self.prior[1] = (j+self.smooth)/denominator
        self.prior[2] = (s+self.smooth)/denominator
        print("The e j s prior probabilities are ", self.prior)
        
    # ---------- Q2/Q3: Estimate the class conditional probability (multinomial parameter) ---------- # 
    def get_theta(self):        # get theta_e _j _s
        numerator = denominator = np.array([0,0,0]).astype('float')
        for set in self.train:
            file = open(self.path+set,'r')
            index = set[0]      # e j s index
            for line in file.readlines():
                for c in line:
                    if '\n' in c:
                        continue
                    else:
                        self.alpha.loc[index,c] += 1    #load characters dist in df
            file.close()
        print(self.alpha)
        denominator = self.alpha.apply(lambda x: x.sum(),axis=1) + (27*self.smooth)
        print(denominator)
        self.alpha += self.smooth
        for i in range(0,3):
            self.alpha.loc[self.lang[i],:] /= denominator[self.lang[i]]
        print(self.alpha)
        
        
            
        

In [76]:
# Q1
naive_bayes = LangId()
naive_bayes.init_sets()
naive_bayes.get_prior()

The e j s prior probabilities are  [0.33333333 0.33333333 0.33333333]


In [77]:
naive_bayes.get_theta()

      a    b    c    d     e    f    g    h     i    j  ...    r     s     t  \
e   910  168  325  332  1594  286  264  714   838   21  ...  814  1001  1212   
j  1885  155   78  246   861   55  200  454  1388   33  ...  612   603   815   
s  1695  133  608  644  1845  139  116   73   808  107  ...  961  1066   577   

      u    v    w   x    y    z        
e   403  140  234  17  209    9  2712  
j  1010    3  282   0  202  110  1766  
s   546   95    1  40  127   43  2728  

[3 rows x 27 columns]
e    15132.5
j    14309.5
s    16215.5
dtype: float64
          a         b         c         d         e         f         g  \
e  0.060169  0.011135  0.021510  0.021973  0.105369  0.018933  0.017479   
j  0.131766  0.010867  0.005486  0.017226  0.060205  0.003879  0.014012   
s  0.104560  0.008233  0.037526  0.039746  0.113811  0.008603  0.007184   

          h         i         j  ...         r         s         t         u  \
e  0.047216  0.055411  0.001421  ...  0.053825  0.066182  0.0

In [ ]:
# Q2 Q3
theta_e = naive_bayes.alpha.loc['e',:]
theta_j = naive_bayes.alpha.loc['j',:]
theta_s = naive_bayes.alpha.loc['s',:]
print(theta_e)
for index, row in theta_j.iteritems():
    print("{} & {} & {} \\\ ".format(index, theta_j[index], theta_s[index]))    # print with latex format

In [88]:
# Q4 
def get_dist(filename):
        alpha_test = {"a":0, "b":0, "c":0, "d":0, "e":0, "f":0, "g":0, "h":0, "i":0, "j":0, "k":0, "l":0, "m":0, 
        "n":0, "o":0, "p":0, "q":0, "r":0, "s":0, "t":0, "u":0, "v":0, "w":0, "x":0, "y":0, "z":0, " ":0}
        file = open(naive_bayes.path+filename,'r')
        for line in file.readlines():
            for c in line:
                if '\n' in c:
                    continue
                else:
                    alpha_test[c] += 1    #load characters dist in df
        file.close()
        return alpha_test

test_dict = get_dist('e10.txt')
print(test_dict)

{'a': 164, 'b': 32, 'c': 53, 'd': 57, 'e': 311, 'f': 55, 'g': 51, 'h': 140, 'i': 140, 'j': 3, 'k': 6, 'l': 85, 'm': 64, 'n': 139, 'o': 182, 'p': 53, 'q': 3, 'r': 141, 's': 186, 't': 225, 'u': 65, 'v': 31, 'w': 47, 'x': 4, 'y': 38, 'z': 2, ' ': 498}


In [86]:
# Q5
def get_loglikelihood(test_dict):
        res = np.zeros(3)
        for i in range(0,3):
            for key in test_dict.keys():
                    res[i] += test_dict[key] * np.log10(naive_bayes.alpha.loc[naive_bayes.lang[i],key])   # Sigma log likelihood
        return res

log_llh = get_loglikelihood(test_dict)
print(log_llh)

[-3405.67889149 -3809.38498463 -3677.29386843]


In [89]:
# Q7
conf_matrix = np.zeros((3,3))
for col, l in enumerate(naive_bayes.lang):
    for i in range(10,20):
        res1 = get_dist(l+str(i)+".txt")
        logll = get_loglikelihood(res1)
        row = np.argmax(logll)
        conf_matrix[row,col] += 1
print(conf_matrix)

[[10.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 10.]]


In [91]:
# Q8
import random
file_test = "e10.txt"
file = open(naive_bayes.path+file_test,'r')
text = list(file.read())
random.shuffle(text)
shuff_text = ''.join(text)
file.close()
file_shuffle= open(naive_bayes.path+"shuffle_e10.txt",'w')
file_shuffle.write(shuff_text)
file_shuffle.close()
shuffle_res = get_dist("shuffle_e10.txt")
log_shuffle = get_loglikelihood(shuffle_res)
print(log_shuffle)

[-3405.67889149 -3809.38498463 -3677.29386843]
